## Setting Up Your Python Environment

In [1]:
# %%capture
# # Install PyTorch with CUDA
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# # Install additional dependencies
# !pip install pandas nobuco tensorflowjs

# # Install utility packages
# !pip install cjm_yolox_pytorch

# Downgrade tensorflow-related packages for nobuco 
# %pip install "keras<3.0.0" "tensorflow<2.16" "tensorflow-decision-forests<1.10.0"

## Importing the Required Dependencies

In [3]:
# Import Python Standard Library dependencies
import json
from pathlib import Path

# Import YOLOX package
from cjm_yolox_pytorch.model import build_model
from cjm_yolox_pytorch.inference import YOLOXInferenceWrapper

# Import the pandas package
import pandas as pd

# Import PyTorch dependencies
import torch

# Import Nobuco dependencies
from nobuco import pytorch_to_keras, ChannelOrder

# Import TensorFlow.js dependencies
from tensorflowjs import converters, quantization

2024-09-26 11:55:13.644174: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-26 11:55:13.666974: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-26 11:55:13.666993: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-26 11:55:13.667618: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-26 11:55:13.671812: I tensorflow/core/platform/cpu_feature_guar

## Setting Up the Project

### Set the Directory Paths

In [4]:
# The name for the project
project_name = f"pytorch-yolox-object-detector"

# The path for the project folder
project_dir = Path(f"./{project_name}/")

# Create the project directory if it does not already exist
project_dir.mkdir(parents=True, exist_ok=True)

# The path to the checkpoint folder
checkpoint_dir = Path(project_dir/f"2024-09-21_20-57-52")

pd.Series({
    "Project Directory:": project_dir,
    "Checkpoint Directory:": checkpoint_dir,
}).to_frame().style.hide(axis='columns')

Project Directory:,pytorch-yolox-object-detector
Checkpoint Directory:,pytorch-yolox-object-detector/2024-09-21_20-57-52


## Loading the Checkpoint Data

### Load the Colormap

In [5]:
# The colormap path
colormap_path = list(checkpoint_dir.glob('*colormap.json'))[0]

# Load the JSON colormap data
with open(colormap_path, 'r') as file:
        colormap_json = json.load(file)

# Convert the JSON data to a dictionary        
colormap_dict = {item['label']: item['color'] for item in colormap_json['items']}

# Extract the class names from the colormap
class_names = list(colormap_dict.keys())

# Make a copy of the colormap in integer format
int_colors = [tuple(int(c*255) for c in color) for color in colormap_dict.values()]

### Load the Normalization Statistics

In [6]:
# The normalization stats path
norm_stats_path = checkpoint_dir/'norm_stats.json'

# Read the normalization stats from the JSON file
with open(norm_stats_path, "r") as f:
    norm_stats_dict = json.load(f)

# Convert the dictionary to a tuple
norm_stats = (norm_stats_dict["mean"], norm_stats_dict["std_dev"])

# Print the mean and standard deviation
pd.DataFrame(norm_stats)

,0,1,2
0,0.5,0.5,0.5
1,1.0,1.0,1.0


### Load the Model Checkpoint

In [7]:
# The model checkpoint path
checkpoint_path = list(checkpoint_dir.glob('*.pth'))[0]

# Load the model checkpoint onto the CPU
model_checkpoint = torch.load(checkpoint_path, map_location='cpu')

/home/innom-dt/mambaforge/envs/pytorch-env/lib/python3.10/site-packages/nobuco/trace/trace.py:366: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  outputs = orig_method(*args,

### Load the Trained YOLOX Model

In [8]:
# Select the YOLOX model configuration
model_type = checkpoint_path.stem

# Create a YOLOX model with the number of output classes equal to the number of class names
model = build_model(model_type, len(class_names))

# Initialize the model with the checkpoint parameters and buffers
model.load_state_dict(model_checkpoint)

The file ./pretrained_checkpoints/yolox_tiny.pth already exists and overwrite is set to False.


<All keys matched successfully>

## Converting the Model to TensorFlow

### Prepare the Model for Inference

In [9]:
# Convert the normalization stats to tensors
mean_tensor = torch.tensor(norm_stats[0]).view(1, 3, 1, 1)
std_tensor = torch.tensor(norm_stats[1]).view(1, 3, 1, 1)

# Set the model to evaluation mode
model.eval();

# Wrap the model with preprocessing and post-processing steps
wrapped_model = YOLOXInferenceWrapper(model, 
                                      mean_tensor, 
                                      std_tensor, 
                                      scale_inp=True, # Scale input values from the rang [0,255] to [0,1]
                                      channels_last=False, # Have the model expect input in channels-last format
                                      run_box_and_prob_calculation=False # Enable or disable post-processing steps
                                     )

### Prepare the Input Tensor

In [10]:
# input_tensor = torch.randn(1, 224, 224, 3)
input_tensor = torch.randn(1, 3, 224, 224)

### Convert the PyTorch Model to Keras 

In [11]:
keras_model = pytorch_to_keras(
    wrapped_model, 
    args=[input_tensor],
    outputs_channel_order=ChannelOrder.PYTORCH, 
)

[Nobuco] Tracing (DONE): 365 ops [00:00]
[Nobuco] Converting: |██▋                                                             | 12/285 ops [00:00]

2024-09-26 11:55:16.609330: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-26 11:55:16.614279: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[Nobuco] Converting (DONE): |████████████████████████████████████████████████████████| 285/285 ops [00:02]
Legend:
    Green — conversion successful
    Yellow — conversion imprecise
    Red — conversion failed
    Red — no converter found
    Bold — conversion applied directly
    * — subgraph reused
    Tensor — this output is not dependent on any of subgraph's input tensors
    Tensor — this input is a parameter / constant
    Tensor — this tensor is useless

YOLOXInferenceWrapper[cjm_yolox_pytorch.inference](float32_0<1,3,224,224>) -> float32_661<1,1029,24>
 │  __truediv__[torch.Tensor](float32_0<1,3,224,224>, 255.0) -> float32_1<1,3,224,224>
 │   └· div[TensorBase](float32_0<1,3,224,224>, 255.0) -> float32_1<1,3,224,224>
 │  __sub__[torch.Tensor](float32_1<1,3,224,224>, float32_2<1,3,1,1>) -> float32_3<1,3,224,224>
 │   └· sub[TensorBase](float32_1<1,3,224,224>, float32_2<1,3,1,1>) -> float32_3<1,3,224,224>
 │  __truediv__[torch.Tensor](float32_3<1,3,224,224>, float32_4<1,3,1,1>) 

### Save the Keras Model in SavedModel format

In [12]:
# Set the folder path for the SavedModel files
savedmodel_dir = Path(f"{checkpoint_dir}/{colormap_path.stem.removesuffix('-colormap')}-{model_type}-tf")
# Save the TensorFlow model to disk
keras_model.save(savedmodel_dir, save_format="tf")

INFO:tensorflow:Assets written to: pytorch-yolox-object-detector/2024-09-21_20-57-52/hagrid-sample-30k-384p-yolox_tiny-tf/assets


INFO:tensorflow:Assets written to: pytorch-yolox-object-detector/2024-09-21_20-57-52/hagrid-sample-30k-384p-yolox_tiny-tf/assets


## Exporting the Model to TensorFlow.js

In [13]:
# Set the path for TensorFlow.js model files
tfjs_model_dir = f"{savedmodel_dir}js-uint8"

# Convert the TensorFlow SavedModel to a TensorFlow.js Graph model
converters.convert_tf_saved_model(saved_model_dir=str(savedmodel_dir), 
                                  output_dir=tfjs_model_dir, 
                                  quantization_dtype_map={quantization.QUANTIZATION_DTYPE_UINT8:True}
                                 )

2024-09-26 11:55:41.798924: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-26 11:55:41.802201: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-09-26 11:55:41.802305: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-09-26 11:55:41.802476: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-26 11:55:41.806181: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2